In [20]:
import numpy as np
import pandas as pd
import scipy.io
import networkx as nx
from dataset_loader import SPD_dataset
%run Pipeline_for_SPD_euclid.ipynb

Populating the interactive namespace from numpy and matplotlib


In [5]:
labels = pd.read_csv("data/targets_final.csv", sep=";").reset_index(drop=True)

idx_to_label = ["C", "D", "E", "DE", "NE", "ND", "TLE"]
label_to_idx = {l : i for i, l in enumerate(idx_to_label)}

for col in labels.columns[1:]:
    labels[col] = labels[col].apply(lambda x: label_to_idx[x] if x in label_to_idx else np.nan)

In [6]:
# labels

In [7]:
patients = labels['patient_number']

In [8]:
data = []
ddir = './data/clean_last/'
for l in patients:
    X_new = pd.read_csv(ddir + '{}.csv'.format(l), index_col=0)
    data.append(np.array(X_new))

In [71]:
def get_spd_dataset(data, threshold, lam = 0.1):
    res =  map(lambda p: create_spd(p, threshold, lam), data)
    return np.stack(res, axis=0)

def create_spd(X, threshold, lam = 0.1):
    N = X.shape[0]
    binary_matrix = (abs(X) > threshold).astype(int)
#     binary_matrix = np.abs(X)
    G = nx.from_numpy_array(binary_matrix)
    X_lp = nx.laplacian_matrix(G, weight = np.abs(X))
    return X_lp.toarray() + lam*np.eye(N)

def create_dataset(problem, threshold):
    # Take proper indices
    idx = labels[problem].notnull()
    
    # Create matrices
    X = [data[i] for i in np.where(idx == True)[0]]
    X = get_spd_dataset(X, threshold)
    
    y = labels[idx][problem]
    if problem[0] =='E':
        pos_label = label_to_idx["E"]
    elif problem[:2] =='DE':
        pos_label = label_to_idx["DE"]
    elif problem[0] =='D':
        pos_label = label_to_idx["D"]
    elif problem[:3] == 'TLE':
        pos_label = label_to_idx["TLE"]
    
    return X, y, pos_label

In [66]:
# np.linalg.eig(x[22])
x[22]

array([[        nan, -0.08366427, -0.08132803, ..., -0.06277338,
        -0.08509858, -0.08990722],
       [-0.08366427,         nan, -0.82096206, ..., -0.42091054,
        -0.61782086, -0.59879208],
       [-0.08132803, -0.82096206,         nan, ..., -0.28489815,
        -0.50342369, -0.45746714],
       ...,
       [-0.06277338, -0.42091054, -0.28489815, ...,         nan,
        -0.79918162, -0.64027611],
       [-0.08509858, -0.61782086, -0.50342369, ..., -0.79918162,
                nan, -0.90753445],
       [-0.08990722, -0.59879208, -0.45746714, ..., -0.64027611,
        -0.90753445,         nan]])

In [63]:
# x[0]
for i in range(x.shape[0]):
    if sum(np.linalg.eig(x[i])[0] <= 0) > 0:
        print(i)
        break
print('ok')

LinAlgError: Array must not contain infs or NaNs

In [50]:
pipe = SPD_pipeline(0.1, 0.1)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=random_state)

grid_cv_lr = pipe.get_lr_grid(cv, scoring, random_state=random_state, n_jobs=1,
                         lr_c_l=[10 ** i for i in range(-4, 2, 2)],
                         metric = ['riemann', 'logeuclid', 'euclid'])

grid_cv_lr.fit(x, np.array(y))

Fitting 50 folds for each of 9 candidates, totalling 450 fits


ValueError: Covariance matrices must be positive definite. Add regularization to avoid this error.